# 导入必要的库

In [10]:
import mindnlp
import mindspore
from mindnlp import core
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

In [11]:
import json

# 数据集预处理

In [12]:
import json

def process_classification(path):
    data = json.load(open(path, "r", encoding="utf-8"))
    processed = []
    for item in data:
        # 构造 instruction, input, output
        instruction = "请阅读以下评论，并判断最后一条评论是否是讽刺性的，并给出讽刺类型。"
        input_text = "\n".join(item["comments"])
        output_text = f"这是讽刺，类型是 {item['classification']}。解析：{item['explanation']}"
        processed.append({
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        })
    return processed

def process_understanding(path):
    data = json.load(open(path, "r", encoding="utf-8"))
    processed = []
    for item in data:
        instruction = f"请回答关于以下评论的问题：{item['question']}"
        input_text = "\n".join(item["comments"])
        output_text = item["qa analysis"]
        processed.append({
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        })
    return processed

def process_roleplay(path):
    data = json.load(open(path, "r", encoding="utf-8"))
    processed = []
    for item in data:
        input_text = "\n".join(item["comments"])
        instruction = "请模仿人类评论者的风格，生成一条讽刺性评论。"
        # 合并角色 A~D 的讽刺评论作为输出候选
        roleplay = item.get("roleplay", {})
        outputs = [v for k,v in roleplay.items() if k in ["A","B","C","D"] and v]
        if not outputs:
            continue
        output_text = "可能的讽刺评论示例：\n" + "\n".join(outputs)
        processed.append({
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        })
    return processed


if __name__ == "__main__":
    # 路径替换成你自己的文件路径
    cls_data = process_classification("classification.json")
    und_data = process_understanding("understanding.json")
    rp_data  = process_roleplay("roleplay.json")

    all_data = cls_data + und_data + rp_data

    with open("train.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"共处理 {len(all_data)} 条数据，已保存到 train.json")

共处理 3752 条数据，已保存到 train.json


# 数据加载

In [13]:
# -------------------------
# Step 2: 数据加载
# -------------------------
df = pd.read_json("./train.json")
ds = Dataset.from_pandas(df)

# 实例化分词器

In [ ]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', 
                                          use_fast=False,
                                          trust_remote_code=True)
tokenizer

LlamaTokenizerFast(name_or_path='./DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False, lstrip=F

In [15]:
def process_func(example):
    MAX_LENGTH = 384
    instruction = tokenizer(
        f"<|im_start|>system\n你是一个讽刺评论生成AI<|im_end|>\n<|im_start|>user\n{example['instruction']}\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)

    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_id = ds.map(process_func, remove_columns=ds.column_names)


Map: 100%|██████████| 3752/3752 [00:05<00:00, 635.30 examples/s]


In [16]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一个讽刺评论生成AI<|im_end|>\n<|im_start|>user\n请阅读以下评论，并判断最后一条评论是否是讽刺性的，并给出讽刺类型。\n水里根本没有鱼，不然我为什么钓不到？\n钓鱼佬永不空军！！！[大笑]\n你是对的\n唯心主义者\n那很有生活了\n钓不到怎么不找找自己的问题？\n菜市场有\n你是对的\n你是对的\n应该是打窝没打好，要不再试试？[doge]\n你得通过实验去证明啊，抽水就是个好办法[doge_金箍]<|im_end|>\n<|im_start|>assistant\n这是讽刺，类型是 fact。解析：该组评论讨论的话题为钓鱼的问题，具体是围绕“水里有没有鱼”以及“为什么钓不到鱼”展开的。最后一条评论讽刺的对象是最初提出“水里根本没有鱼，不然我为什么钓不到？”这一观点的人。该评论通过事实讽刺(fact)的方式，表面上看似认真地建议通过“抽水”来验证水里是否有鱼，但实际上这种做法是非常不切实际且荒谬的。评论者利用这种极端的方式来讽刺原观点的逻辑问题：仅凭自己钓不到鱼就断定水里没有鱼，显然是站不住脚的。通过提出一个明显不合理但理论上可行的实验方法（抽水），评论者突出了原观点的荒谬性，并暗示其推理过于简单和片面，从而达到讽刺效果。<｜end▁of▁sentence｜>'

# 模型加载

In [ ]:
# 加载基础模型
model = AutoModelForCausalLM.from_pretrained('deepseek-ai//DeepSeek-R1-Distill-Qwen-1.5B',
                                             ms_dtype=mindspore.bfloat16, device_map=0)

# 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()

# 微调前推理
进行模型推理查看效果

In [18]:
# -------------------------
# 微调前推理
# -------------------------

# host to device
model = model.npu()

prompt = "请针对下面的评论生成一条讽刺评论：\n水里根本没有鱼，不然我为什么钓不到？"
inputs = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "你是一个讽刺评论生成AI"},
        {"role": "user", "content": prompt}
    ],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="ms",   # MindSpore Tensor
    return_dict=True
).to("cuda")

gen_kwargs = {"max_length": 512, "do_sample": True, "top_k": 5}

with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1]:]
    print("微调前推理结果：")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


微调前推理结果：
嗯，用户给了一个评论：“水里根本没有鱼，不然我为什么钓不到？”然后让我生成一条讽刺评论。首先，我需要理解这个评论的意思。用户是在说，如果水里真的有鱼，为什么他钓不到，可能是因为鱼太小或者水太深，或者他钓的线太短。

接下来，我得考虑如何用讽刺的方式回应。讽刺通常是指带有贬义或暗示错误的评论，用来指出问题所在。所以，我需要找到一个反问的方式，或者指出原因，但带有讽刺的语气。

可能的思路是：“你为什么钓不到？难道水里没有鱼吗？” 这样直接反问，可能显得有些讽刺，因为如果水里有鱼，为什么钓不到。但可能用户希望更深入一点，指出原因。

另一种可能是：“你钓不到的原因，可能是因为水太深，或者鱼太小，或者你的线太短了。” 这样指出原因，但用讽刺的语气，比如“太深了”可能让人觉得奇怪，但讽刺效果也不错。

再想想，用户可能希望这条评论能引发读者的思考，或者让读者反思自己的行为。所以，可能需要加入一些隐喻，比如鱼在水里，而你钓不到，可能是因为你错过了什么。

或者，可以反问：“你为什么钓不到？难道你没有学会如何钓鱼？” 这样指出问题，但可能不够讽刺，因为讽刺通常需要更强烈的反问或隐喻。

再进一步，可能用反问句，但带有讽刺的语气，比如：“你钓不到的原因，可能是因为水里没有鱼，或者你钓的线太短了。” 这样指出原因，但可能不够讽刺。

或者，更直接的讽刺，比如：“你钓不到的原因，可能是因为你钓的线太短了，或者水太深了，或者鱼太小了。” 这样指出原因，但可能不够讽刺。

可能需要更隐喻的讽刺，比如：“你钓不到的原因，可能是因为你钓的鱼太小了，或者水太深了，或者你钓的线太短了。” 这样指出原因，但可能不够讽刺。

或者，更直接的讽刺，比如：“你钓不到的原因，可能是因为你钓的线太短了


# 配置lora

In [19]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 9,232,384 || all params: 1,786,320,384 || trainable%: 0.5168


# 训练参数与微调训练

In [20]:
# -------------------------
# Step 6: 训练参数
# -------------------------
args = TrainingArguments(
    output_dir="./output_lora_sarcasm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=5,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()

Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[WARNING] PRE_ACT(51327,ffff9aee0640,python):2025-09-11-13:54:28.114.292 [mindspore/ccsrc/memory/mem_pool/abstract_dynamic_mem_pool.cc:1099] FreeIdleMemsByEagerFree] Eager free count : 1, free memory : 3517215232, real free : 3514826752, not free : 2388480.


.

Step,Training Loss
10,3.259500
20,2.722600
30,2.431500
40,2.309000
50,2.139700
60,2.173800
70,2.221700
80,2.094800
90,2.128500
100,2.025900


TrainOutput(global_step=564, training_loss=1.9023796531325536, metrics={'train_runtime': 2065.7749, 'train_samples_per_second': 5.449, 'train_steps_per_second': 0.273, 'total_flos': 3.5074245140066304e+16, 'train_loss': 1.9023796531325536, 'epoch': 3.0})

# 微调后推理

In [21]:
# -------------------------
# Step 7: 微调后推理
# -------------------------
mode_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
lora_path = "./output_lora_sarcasm/checkpoint-564"  # 修改为保存的 checkpoint 路径

model = AutoModelForCausalLM.from_pretrained(
    mode_path, ms_dtype=mindspore.bfloat16, trust_remote_code=True
).eval()
model = PeftModel.from_pretrained(model, model_id=lora_path)
model = model.npu()

prompt = "请针对下面的评论生成一条讽刺评论：\n水里根本没有鱼，不然我为什么钓不到？"
inputs = tokenizer.apply_chat_template(
    [{"role": "system", "content": "你是一个讽刺评论生成AI"},
     {"role": "user", "content": prompt}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="ms",
    return_dict=True
).to("cuda")

gen_kwargs = {"max_length": 512, "do_sample": True, "top_k": 5}
with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1]:]
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


可能的讽刺评论示例：
哈哈，水里根本没有鱼，不然我钓不到鱼啊！真是个‘奇才’啊！😄
对啊，如果水里没有鱼，那钓鱼就太简单了，根本不需要任何努力！👍
没错，水里根本不可能有鱼，不然我怎么能在水里找到鱼呢？🙄
看来我之前对水里的鱼一无所知，真是个‘新秀’啊！😏
